In [292]:
import numpy as np
import pandas as pd
import plotly as py
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_distplot
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import ProbPlot
import missingno as msno
import json
from pandas.io.json import json_normalize
py.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [293]:
#write a function to read json files with list of list of dictionaries
def jsonreader(file):
    with open(file) as json_data:
        data = json.load(json_data)
    
    df = pd.DataFrame()
    
    for i in range(len(data)):
        df = df.append(json_normalize(data[i]), ignore_index = True)
        
    return df

In [296]:
#load in the data
flight = jsonreader('./Datasets/flights_250days_march17.json')





KeyboardInterrupt: 

In [297]:
flightdf = flight.copy()

In [298]:
flightdf.dtypes

airline            object
arrival            object
date               object
departure          object
error              object
flight duration    object
plane              object
plane code         object
scrape_date        object
stops              object
ticket price       object
timings            object
dtype: object

In [301]:
flightdf.isnull().sum()

airline            1748
arrival            1748
date               1748
departure          1748
flight duration    1748
plane              1748
plane code         1748
scrape_date        1748
stops              1748
ticket price       1748
timings            1748
dtype: int64

In [300]:
flightdf.drop(['error'], axis = 1, inplace = True)

In [306]:
#replace '' with nan
flightdf.replace('', np.nan, inplace = True)

In [305]:
#check if there is any missingness
flightdf.isnull().sum()

airline            1802
arrival            1748
date               1748
departure          1748
flight duration    1748
plane              4346
plane code         4346
scrape_date        1748
stops              1748
ticket price       1748
timings            1748
dtype: int64

In [304]:
flightdf[flightdf.isnull().any(axis = 1)]

,airline,arrival,date,departure,flight duration,plane,plane code,scrape_date,stops,ticket price,timings
11,Delta,", Los Angeles",03/18/2018,", New York",0 days 6 hours 13 minutes,NaN,NaN,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'New York, NY (JFK-John..."
24,Delta,", Los Angeles",03/18/2018,", New York",0 days 6 hours 17 minutes,NaN,NaN,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'New York, NY (JFK-John..."
35,Delta,", Chicago",03/18/2018,", New York",0 days 2 hours 42 minutes,NaN,NaN,03/17/2018,Nonstop,161.30,"[{'departure_airport': 'New York, NY (JFK-John..."
52,Delta,", Chicago",03/18/2018,", New York",0 days 3 hours 0 minutes,NaN,NaN,03/17/2018,Nonstop,207.30,"[{'departure_airport': 'New York, NY (JFK-John..."
100,Delta,", Boston",03/18/2018,", New York",0 days 1 hours 16 minutes,NaN,NaN,03/17/2018,Nonstop,415.30,"[{'departure_airport': 'New York, NY (JFK-John..."
102,Delta,", Boston",03/18/2018,", New York",0 days 1 hours 20 minutes,NaN,NaN,03/17/2018,Nonstop,415.30,"[{'departure_airport': 'New York, NY (JFK-John..."
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,Delta,", New York",03/18/2018,", Los Angeles",0 days 5 hours 34 minutes,NaN,NaN,03/17/2018,Nonstop,952.60,"[{'departure_airport': 'Los Angeles, CA (LAX-L..."
128,Delta,", New York",03/18/2018,", Los Angeles",1 days 5 hours 35 minutes,NaN,NaN,03/17/2018,Nonstop,952.60,"[{'departure_airport': 'Los Angeles, CA (LAX-L..."
141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [316]:
flightdf = flightdf[flightdf['airline'].notnull()]

In [317]:
flightdf.isnull().sum()

airline               0
arrival               0
date                  0
departure             0
flight duration       0
plane              2598
plane code         2598
scrape_date           0
stops                 0
ticket price          0
timings               0
dtype: int64

In [209]:
# [df[df[i] == ''] for i in df.columns]

In [318]:
#cleaned out ','
flightdf['arrival'] = flightdf['arrival'].map(lambda x: x.replace(',', ''))
flightdf['departure'] = flightdf['departure'].map(lambda x: x.replace(',', ''))



In [319]:
#convert flight ticket price from string to float 
flightdf['ticket price'] = flightdf['ticket price'].astype('float')

In [320]:
flightdf

,airline,arrival,date,departure,flight duration,plane,plane code,scrape_date,stops,ticket price,timings
0,Spirit Airlines,Los Angeles,03/18/2018,New York,1 days 11 hours 35 minutes,AIRBUS INDUSTRIE A321 SHARKLETS,32B,03/17/2018,1 Stop,345.98,"[{'departure_airport': 'New York, NY (LGA-LaGu..."
1,American Airlines,Los Angeles,03/18/2018,New York,1 days 19 hours 36 minutes,Boeing 737-800,738,03/17/2018,2 Stop,400.81,"[{'departure_airport': 'New York, NY (JFK-John..."
2,American Airlines,Los Angeles,03/18/2018,Newark,1 days 8 hours 26 minutes,Embraer 175,E75,03/17/2018,1 Stop,410.00,"[{'departure_airport': 'Newark, NJ (EWR-Libert..."
3,JetBlue Airways,Los Angeles,03/18/2018,New York,0 days 9 hours 39 minutes,Airbus A320,320,03/17/2018,1 Stop,451.00,"[{'departure_airport': 'New York, NY (JFK-John..."
4,Delta,Los Angeles,03/18/2018,New York,0 days 6 hours 20 minutes,Boeing 767,764,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'New York, NY (JFK-John..."
5,United,Los Angeles,03/18/2018,Newark,0 days 6 hours 4 minutes,Boeing 777,777,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'Newark, NJ (EWR-Libert..."
6,United,Los Angeles,03/18/2018,Newark,0 days 6 hours 9 minutes,Boeing 757,757,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'Newark, NJ (EWR-Libert..."
7,Delta,Los Angeles,03/18/2018,New York,1 days 6 hours 16 minutes,Boeing 757-200,75W,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'New York, NY (JFK-John..."
8,Alaska Airlines,Los Angeles,03/18/2018,New York,0 days 6 hours 25 minutes,Airbus A321,321,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'New York, NY (JFK-John..."
9,United,Los Angeles,03/18/2018,Newark,0 days 6 hours 6 minutes,Boeing 777,777,03/17/2018,Nonstop,476.30,"[{'departure_airport': 'Newark, NJ (EWR-Libert..."
